In [ ]:
import numpy as np
from dataclasses import dataclass

@dataclass
class MotorParams:
    Rs: float = 0.05          # stator resistance per phase [ohm]
    Ls: float = 0.0006        # stator inductance per phase [H]
    Ke: float = 0.07          # back‑EMF constant [V·s/rad]
    Kt: float = 0.07          # torque constant [N·m/A]
    J: float = 0.01           # inertia [kg·m^2]
    B: float = 1e-3           # viscous friction [N·m·s]
    pole_pairs: int = 4

@dataclass
class InvParams:
    Vdc: float = 400.0
    m: float = 0.9            # modulation index (0..1)

@dataclass
class SimConfig:
    fs: float = 20000.0       # electrical sampling rate [Hz]
    t_end: float = 2.0        # seconds
    speed_mech_rpm: float = 1500.0
    load_torque: float = 10.0

@dataclass
class FaultConfig:
    kind: str = "healthy"     # 'healthy'|'interturn'|'open'|'sw_short'|'bearing'|'eccentric'
    phase: str = "a"          # affected phase for certain faults
    severity: float = 0.3     # 0..1 generic severity
    bearing_freq: float = 150.0  # Hz ripple

_phase_idx = {"a":0, "b":1, "c":2}

class MotorInverterSim:
    def _init_(self, mp: MotorParams, ip: InvParams, sc: SimConfig, fc: FaultConfig):
        self.mp, self.ip, self.sc, self.fc = mp, ip, sc, fc
        self.dt = 1.0 / sc.fs
        self.n = int(sc.t_end * sc.fs)
        self.theta = 0.0
        self.omega = (sc.speed_mech_rpm * 2*np.pi/60.0) * mp.pole_pairs  # electrical rad/s
        self.i = np.zeros(3)  # ia, ib, ic

    def _bemf(self, theta):
        # back‑EMFs per phase with optional eccentricity (amplitude modulation)
        ecc = 1.0 + (0.4*self.fc.severity if self.fc.kind == 'eccentric' else 0.0)*np.sin(5*theta)
        Ke = self.mp.Ke * ecc
        ea = Ke*self.omega*np.sin(theta)
        eb = Ke*self.omega*np.sin(theta - 2*np.pi/3)
        ec = Ke*self.omega*np.sin(theta + 2*np.pi/3)
        return np.array([ea, eb, ec])

    def _spwm_vphase(self, theta):
        # sinusoidal PWM approximation (phase‑neutral voltages)
        ma = self.ip.m
        V = 0.5*self.ip.Vdc*ma
        va = V*np.sin(theta)
        vb = V*np.sin(theta - 2*np.pi/3)
        vc = V*np.sin(theta + 2*np.pi/3)
        return np.array([va, vb, vc])

    def _apply_inverter_faults(self, v):
        if self.fc.kind == 'sw_short':
            k = _phase_idx[self.fc.phase]
            # clamp affected phase to ±Vdc/2 depending on sign of sine reference
            v[k] = 0.5*self.ip.Vdc*np.sign(v[k])
        return v

    def _phase_params(self):
        Rs = np.full(3, self.mp.Rs)
        Ls = np.full(3, self.mp.Ls)
        if self.fc.kind == 'interturn':
            k = _phase_idx[self.fc.phase]
            Ls[k] *= (1.0 - 0.7*self.fc.severity)  # reduce L
            Rs[k] *= (1.0 + 1.5*self.fc.severity)  # increase copper loss
        if self.fc.kind == 'open':
            k = _phase_idx[self.fc.phase]
            Rs[k] *= (1e6)  # effectively open circuit
        return Rs, Ls

    def step(self):
        # compute references
        v = self._spwm_vphase(self.theta)
        v = self._apply_inverter_faults(v.copy())
        e = self._bemf(self.theta)
        Rs, Ls = self._phase_params()
        # di/dt per phase (explicit Euler)
        di = (v - Rs*self.i - e)/Ls
        self.i += di * self.dt
        # electromechanical update
        Te = self.mp.Kt*( self.i[0]*np.sin(self.theta)
                         + self.i[1]*np.sin(self.theta - 2*np.pi/3)
                         + self.i[2]*np.sin(self.theta + 2*np.pi/3) )
        Tl = self.sc.load_torque
        if self.fc.kind == 'bearing':
            Tl *= (1.0 + 0.6*self.fc.severity*np.sin(2*np.pi*self.fc.bearing_freq*self.t))
        domega = (Te - Tl - self.mp.B*self.omega)/self.mp.J
        self.omega += domega*self.dt
        self.theta += self.omega*self.dt
        self.t += self.dt

    def run(self):
        self.t = 0.0
        ia, ib, ic, va, vb, vc, om = [], [], [], [], [], [], []
        for _ in range(self.n):
            v = self._spwm_vphase(self.theta)
            v = self._apply_inverter_faults(v.copy())
            e = self._bemf(self.theta)
            Rs, Ls = self._phase_params()
            di = (v - Rs*self.i - e)/Ls
            self.i += di * self.dt
            Te = self.mp.Kt*( self.i[0]*np.sin(self.theta)
                             + self.i[1]*np.sin(self.theta - 2*np.pi/3)
                             + self.i[2]*np.sin(self.theta + 2*np.pi/3) )
            Tl = self.sc.load_torque
            if self.fc.kind == 'bearing':
                Tl *= (1.0 + 0.6*self.fc.severity*np.sin(2*np.pi*self.fc.bearing_freq*self.t))
            domega = (Te - Tl - self.mp.B*self.omega)/self.mp.J
            self.omega += domega*self.dt
            self.theta += self.omega*self.dt
            ia.append(self.i[0]); ib.append(self.i[1]); ic.append(self.i[2])
            va.append(v[0]); vb.append(v[1]); vc.append(v[2])
            om.append(self.omega)
            self.t += self.dt
        return {
            'ia': np.array(ia), 'ib': np.array(ib), 'ic': np.array(ic),
            'va': np.array(va), 'vb': np.array(vb), 'vc': np.array(vc),
            'omega': np.array(om), 'fs': self.sc.fs
        }